# 02 — Core SQL Queries (Revenue & Funnel)
_Date: 2025-09-12_

This notebook assumes a SQLite database at `C:/Users/nazar/OneDrive/Documentos/data`.
Use 'C:/Users/nazar/OneDrive/Documentos/E-commerce Revenue Breakdown & Funnel Metrics/01_exploration.ipynb' to load CSVs into SQLite first.


In [6]:
import sqlite3, pandas as pd, os
DB_PATH = 'C:/Users/nazar/OneDrive/Documentos/data/olist.db'
assert os.path.exists(DB_PATH), f"Database not found: {DB_PATH}. Run src/data_loader.py first."
conn = sqlite3.connect(DB_PATH)
conn.execute('PRAGMA foreign_keys = ON;');
print('Connected to', DB_PATH)


Connected to C:/Users/nazar/OneDrive/Documentos/data/olist.db


## Revenue by Product Category

In [7]:
q_revenue_by_category = '''
SELECT 
  p.product_category_name AS category,
  ROUND(SUM(oi.price), 2) AS total_revenue,
  COUNT(DISTINCT oi.order_id) AS num_orders
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY 1
ORDER BY total_revenue DESC;
'''
df_rev_cat = pd.read_sql_query(q_revenue_by_category, conn)
df_rev_cat.head()


,category,total_revenue,num_orders
0,beleza_saude,1258681.34,8836
1,relogios_presentes,1205005.68,5624
2,cama_mesa_banho,1036988.68,9417
3,esporte_lazer,988048.97,7720
4,informatica_acessorios,911954.32,6689


## Revenue by Customer State

In [8]:
q_revenue_by_state = '''
SELECT 
  c.customer_state AS state,
  ROUND(SUM(oi.price), 2) AS total_revenue,
  COUNT(DISTINCT oi.order_id) AS num_orders
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY 1
ORDER BY total_revenue DESC;
'''
df_rev_state = pd.read_sql_query(q_revenue_by_state, conn)
df_rev_state.head()


,state,total_revenue,num_orders
0,SP,5202955.05,41375
1,RJ,1824092.67,12762
2,MG,1585308.03,11544
3,RS,750304.02,5432
4,PR,683083.76,4998


## Payment Mix & AOV

In [10]:
# Consulta 1: Mezcla de métodos de pago
q_payment_mix = '''
SELECT payment_type,
       COUNT(*) AS payments,
       ROUND(SUM(payment_value), 2) AS revenue
FROM order_payments
GROUP BY 1
ORDER BY revenue DESC;
'''

df_payment = pd.read_sql_query(q_payment_mix, conn)
print(df_payment.head())


# Consulta 2: Average Order Value (AOV)
q_aov = '''
SELECT ROUND(SUM(payment_value) * 1.0 / COUNT(DISTINCT order_id), 2) AS AOV
FROM order_payments;
'''

df_aov = pd.read_sql_query(q_aov, conn)
print(df_aov)


  payment_type  payments      revenue
0  credit_card     76795  12542084.19
1       boleto     19784   2869361.27
2      voucher      5775    379436.87
3   debit_card      1529    217989.79
4  not_defined         3         0.00
      AOV
0  160.99


## Funnel (Created → Approved → Delivered → Invoiced)
Note: Olist does not have web events; we approximate funnel stages using `orders.order_status` and timestamps.

In [11]:
q_funnel = '''
WITH stages AS (
  SELECT 
    COUNT(DISTINCT CASE WHEN order_status IN ('created','approved','invoiced','shipped','delivered') THEN order_id END) AS any_status,
    COUNT(DISTINCT CASE WHEN order_status IN ('created','approved') THEN order_id END) AS created_or_approved,
    COUNT(DISTINCT CASE WHEN order_status = 'approved' THEN order_id END) AS approved,
    COUNT(DISTINCT CASE WHEN order_status = 'invoiced' THEN order_id END) AS invoiced,
    COUNT(DISTINCT CASE WHEN order_status = 'shipped' THEN order_id END) AS shipped,
    COUNT(DISTINCT CASE WHEN order_status = 'delivered' THEN order_id END) AS delivered,
    COUNT(DISTINCT CASE WHEN order_status = 'canceled' THEN order_id END) AS canceled
  FROM orders
)
SELECT * FROM stages;
'''
df_funnel = pd.read_sql_query(q_funnel, conn)
df_funnel


,any_status,created_or_approved,approved,invoiced,shipped,delivered,canceled
0,97906,7,2,314,1107,96478,625


In [12]:
# Derived conversion rates
if not df_funnel.empty:
    s = df_funnel.iloc[0]
    created = max(s.get('created_or_approved', 0), 1)
    approved = max(s.get('approved', 0), 1)
    delivered = max(s.get('delivered', 0), 1)
    invoiced = max(s.get('invoiced', 0), 1)
    shipped = max(s.get('shipped', 0), 1)
    rates = {
        'approved / created': round(s.get('approved',0) / created, 4),
        'invoiced / approved': round(s.get('invoiced',0) / approved, 4),
        'shipped / invoiced': round(s.get('shipped',0) / invoiced, 4),
        'delivered / shipped': round(s.get('delivered',0) / shipped, 4),
        'delivered / created': round(s.get('delivered',0) / created, 4),
    }
    rates
else:
    {}


In [13]:
from datetime import datetime
import pandas as pd
from pathlib import Path

# --- formateo opcional ---
df_payment_fmt = df_payment.copy()
if "revenue" in df_payment_fmt.columns:
    df_payment_fmt["revenue"] = df_payment_fmt["revenue"].map(lambda x: f"${x:,.2f}")

df_aov_fmt = df_aov.copy()
if "AOV" in df_aov_fmt.columns:
    df_aov_fmt["AOV"] = df_aov_fmt["AOV"].map(lambda x: f"${x:,.2f}")

# --- HTML minimalista con algo de estilo ---
css = """
<style>
body { font-family: Arial, sans-serif; margin: 24px; }
h1 { margin-bottom: 0; }
h2 { margin-top: 32px; }
table { border-collapse: collapse; width: 100%; }
th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
th { background: #f5f5f5; }
caption { caption-side: bottom; text-align: left; color: #666; font-size: 12px; padding-top: 8px; }
</style>
"""

html = f"""
<!doctype html>
<html>
<head>
<meta charset="utf-8">
<title>Reporte Olist</title>
{css}
</head>
<body>
  <h1>Reporte Olist</h1>
  <p>Generado: {datetime.now().strftime('%Y-%m-%d %H:%M')}</p>

  <h2>Mix de métodos de pago</h2>
  {df_payment_fmt.to_html(index=False, border=0)}

  <h2>Average Order Value (AOV)</h2>
  {df_aov_fmt.to_html(index=False, border=0)}
</body>
</html>
"""

out_dir = Path(r"C:\Users\nazar\OneDrive\Documentos\data")
out_dir.mkdir(parents=True, exist_ok=True)
out_file = out_dir / "reporte_olist.html"

with open(out_file, "w", encoding="utf-8") as f:
    f.write(html)

print(f"HTML guardado en: {out_file}")


HTML guardado en: C:\Users\nazar\OneDrive\Documentos\data\reporte_olist.html
